In [76]:
import finvizfinance as fz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from finvizfinance.quote import finvizfinance, Statements, Quote
import yfinance as yf
import math


In [77]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)


In [78]:
tickers = [
    "EQIX", "DLR",
    "PWR", "ETN", "ABB",
    "VRT", "TT", "JCI",
    "SMCI", "HPE", "DELL",
    "ANET", "AVGO", "CSCO",
    "MRVL", "COHR", "LITE", "INFN",
    "MU", "ON", "STM", "ADI", "TXN",
    "TSM", "INTC", "ASX", "AMKR", "PLAB",
    "SNPS", "CDNS", "ANSS",
    "STX", "WDC",
    "ASML", "AMAT", "KLAC", "LRCX",
    "NVDA",
    "GOOG", "ADBE",
    "000660.KS", "INDIGO.NS", "NVO", "UNH", "RHM.DE", 
    "VOLTAMP.NS", "ADANIPOWER.NS", "WAAREEENER.BO", "LLY"
]

# tickers = [
#     "NVO", "UNH", "000660.KS", "INDIGO.NS",
#     "TSM", 
#     "ASML",
#     "NVDA",
#     "GOOG", "GOOGL", "ADBE"
# ]

# tickers = ["TSM"]


In [ ]:
bundles= []

for ticker in tickers:
    print(f'scraping: {ticker}')
    try:
        stock = yf.Ticker(ticker)
    except Exception as e:
        print(f'error scraping {ticker}: {e}')
        continue

    stock_fundamentals = stock.info
    
    income_df_q= stock.quarterly_incomestmt
    cashflow_df_q= stock.quarterly_cashflow
    balance_df_q= stock.quarterly_balance_sheet

    income_df_y= stock.incomestmt
    cashflow_df_y= stock.cashflow
    balance_df_y= stock.balancesheet

    bundle= {
        'fund': stock_fundamentals,
        'inc_q': income_df_q,
        'cf_q': cashflow_df_q,
        'bs_q': balance_df_q,
        'inc_y': income_df_y,
        'cf_y': cashflow_df_y,
        'bs_y': balance_df_y
    }
    bundles.append(bundle)


In [92]:

def _to_float(x, default=0.0):
    if x is None:
        return default
    try:
        return float(x)
    except Exception:
        s = str(x).replace(",", "").strip()
        if s.endswith("%"):
            s = s[:-1]
        try:
            return float(s)
        except Exception:
            return default

def _to_pct(x, default=0.0):
    v = _to_float(x, default)
    if v is None:
        return default
    # Yahoo-style decimals (e.g., 0.60) -> %
    return v * 100.0 if -1.5 <= v <= 1.5 else v

def _pos(x, lo, hi):
    if pd.isna(x):
        return 0.0
    if x <= lo: return 0.0
    if x >= hi: return 1.0
    return (x - lo) / (hi - lo)

def _neg(x, lo, hi):
    if pd.isna(x):
        return 0.0
    if x <= lo: return 1.0
    if x >= hi: return 0.0
    return 1.0 - (x - lo) / (hi - lo)

def _series_recentN(df, row_label, n):
    if (df is None) or (row_label not in df.index):
        return [0.0] * n
    row = df.loc[row_label]
    m = min(n, row.shape[0])
    vals = [_to_float(row.iloc[i], 0.0) for i in range(m)]
    while len(vals) < n:
        vals.append(vals[-1] if vals else 0.0)
    return vals

def _weighted_recent(values, weights):
    k = min(len(values), len(weights))
    v = values[:k]; w = weights[:k]
    s = sum(w)
    if s == 0: return sum(v)/k if k else 0.0
    return sum(v[i]*w[i] for i in range(k)) / s

def _safe_cv(series):
    if not series: return None
    mu = sum(series) / len(series)
    if abs(mu) < 1e-9: return None
    var = sum((x - mu)**2 for x in series) / max(1, len(series) - 1)
    from math import sqrt
    return sqrt(var) / abs(mu)

def _blend(recent, long, alpha):
    if pd.isna(recent) and pd.isna(long): 
        return None
    if pd.isna(recent): 
        return long
    if pd.isna(long): 
        return recent
    return alpha*recent + (1-alpha)*long

    # try:
    #     if (recent is None) or (recent is pd.isna(recent)) and (long is None) or (long is pd.isna(long)): return None
    #     if (recent is None) or (recent is pd.isna(recent)): return long
    #     if (long   is None) or (long is pd.isna(long)): return recent
    #     return alpha*recent + (1-alpha)*long
    # except:
    #     import pdb; pdb.set_trace()
    # return None

def _median(xs):
    xs = [x for x in xs if x is not None]
    if not xs: return None
    xs = sorted(xs)
    n = len(xs)
    return xs[n//2] if n%2 else 0.5*(xs[n//2-1]+xs[n//2])


In [93]:
# --- simple_buy_score.py (5 quarters + 4 annual years) -----------------------


# # --------- extraction tuned to: 5 quarters + 4 annual years ----------
# def extract_from_statements(info_dict, income_q, cashflow_q, balance_q, income_y, cashflow_y):
#     # --- Growth (annual 4y: CAGR over 3-year span) ---
#     # Annual columns must be most-recent first: y0, y1, y2, y3

#     income_y= income_q
#     cashflow_y= cashflow_q
    
#     if ('Total Revenue' in income_y.index) and (income_y.shape[1] >= 4):
#         r0 = _to_float(income_y.loc['Total Revenue'].iloc[0])
#         r3 = _to_float(income_y.loc['Total Revenue'].iloc[3])
#         if r3 > 0:
#             yrs = 3
#             rev_growth = ((r0 / r3) ** (1/yrs) - 1) * 100.0
#         else:
#             # fallback to simple YoY if base is zero/invalid
#             r1 = _to_float(income_y.loc['Total Revenue'].iloc[1], 0.0)
#             rev_growth = ((r0 - r1) / r1 * 100.0) if r1 > 0 else _to_pct(info_dict.get('revenueGrowth', 0.0))
#     else:
#         # last resort
#         rev_growth = _to_pct(info_dict.get('revenueGrowth', 0.0))

#     # --- FCF margin (quarters: use last 4 of 5 with linear decay) ---
#     fcf4 = _series_recentN(cashflow_q, 'Free Cash Flow', 4)
#     rev4 = _series_recentN(income_q,   'Total Revenue', 4)
#     fcf_w = _weighted_recent(fcf4, [1.0, 0.75, 0.50, 0.25])
#     rev_w = _weighted_recent(rev4,     [1.0, 0.75, 0.50, 0.25])
#     fcf_margin = (fcf_w / rev_w * 100.0) if rev_w > 0 else 0.0

#     # --- Current Ratio (quarterly latest) ---
#     if ('Current Assets' in balance_q.index) and ('Current Liabilities' in balance_q.index):
#         ca = _to_float(balance_q.loc['Current Assets'].iloc[0])
#         cl = _to_float(balance_q.loc['Current Liabilities'].iloc[0])
#         curr_ratio = (ca / cl) if (cl and cl != 0) else _to_float(info_dict.get('currentRatio', 1.0))
#     else:
#         curr_ratio = _to_float(info_dict.get('currentRatio', 1.0))

#     # --- Debt/Equity (quarterly latest if possible; else Yahoo percent/100) ---
#     if ('Total Debt' in balance_q.index) and ('Stockholders Equity' in balance_q.index):
#         td = _to_float(balance_q.loc['Total Debt'].iloc[0], 0.0)
#         se = _to_float(balance_q.loc['Stockholders Equity'].iloc[0], 0.0)
#         debt_eq = (td / se) if (se > 0 and not math.isnan(td)) else _to_float(info_dict.get('debtToEquity', 0.0)/100)
#     else:
#         debt_eq = _to_float(info_dict.get('debtToEquity', 0.0))/100.0

#     # --- Stability (annual YoY over 3 intervals, take CV) ---
#     rev_yoy_cv = None
#     if ('Total Revenue' in income_y.index) and (income_y.shape[1] >= 4):
#         ry = [_to_float(income_y.loc['Total Revenue'].iloc[i], 0.0) for i in range(4)]
#         yoy = []
#         for i in range(3):  # y0 vs y1, y1 vs y2, y2 vs y3
#             base = ry[i+1]
#             yoy.append(((ry[i] - base)/base * 100.0) if base > 0 else 0.0)
#         rev_yoy_cv = _safe_cv(yoy) if len(yoy) >= 2 else None

#     # --- Moat (annual GM stability across 4 years; CV of GM%) ---
#     gm_cv = None
#     if all(lbl in income_y.index for lbl in ['Gross Profit', 'Total Revenue']) and (income_y.shape[1] >= 4):
#         gp = [_to_float(income_y.loc['Gross Profit'].iloc[i], 0.0) for i in range(4)]
#         rv = [_to_float(income_y.loc['Total Revenue'].iloc[i], 0.0) for i in range(4)]
#         gms = [(gp[i]/rv[i]*100.0) if rv[i] > 0 else 0.0 for i in range(4)]
#         gm_cv = _safe_cv(gms) if len(gms) >= 2 else None

#     # --- R&D intensity (latest annual) ---
#     rd_intensity = None
#     if all(lbl in income_y.index for lbl in ['Research And Development', 'Total Revenue']):
#         rdy = _to_float(income_y.loc['Research And Development'].iloc[0], 0.0)
#         rvy = _to_float(income_y.loc['Total Revenue'].iloc[0], 0.0)
#         rd_intensity = (rdy / rvy * 100.0) if rvy > 0 else 0.0

#     # --- Investment ratio (latest annual: |CapEx| / OCF) ---
#     invest_ratio = None
#     if (cashflow_y is not None) and all(lbl in cashflow_y.index for lbl in ['Capital Expenditure', 'Operating Cash Flow']):
#         capex_y = abs(_to_float(cashflow_y.loc['Capital Expenditure'].iloc[0], 0.0))
#         ocf_y   = _to_float(cashflow_y.loc['Operating Cash Flow'].iloc[0], 0.0)
#         invest_ratio = (capex_y / ocf_y * 100.0) if ocf_y > 0 else 0.0

#     return (rev_growth, fcf_margin, debt_eq, curr_ratio,
#             rev_yoy_cv, gm_cv, rd_intensity, invest_ratio)


def extract_from_statements(info_dict, income_q, cashflow_q, balance_q, income_y, cashflow_y):
    # ---------- TTM (RECENT) ----------
    rev_q = _series_recentN(income_q, 'Total Revenue', 5)
    gp_q  = _series_recentN(income_q, 'Gross Profit', 5)
    rd_q  = _series_recentN(income_q, 'Research And Development', 5)
    capex_q = _series_recentN(cashflow_q, 'Capital Expenditure', 5)
    ocf_q   = _series_recentN(cashflow_q, 'Operating Cash Flow', 5)

    # TTM sums (last 4 quarters)
    rev_ttm = sum(rev_q[:4]); gp_ttm = sum(gp_q[:4]); rd_ttm = sum(rd_q[:4])
    capex_ttm = sum(abs(x) for x in capex_q[:4]); ocf_ttm = sum(ocf_q[:4])

    # Growth (recent): YoY from quarters (Q0 vs Q4 if present)
    rev_ttm_growth_recent = None
    if len(rev_q) >= 5 and rev_q[4] > 0:
        rev_ttm_growth_recent = ((rev_q[0] - rev_q[4]) / rev_q[4]) * 100.0

    # Stability (recent): CV of quarterly YoY over available pairs
    rev_yoy_recent = []
    if len(rev_q) >= 5:
        for i in range(len(rev_q)-4):
            base = rev_q[i+4]
            rev_yoy_recent.append(((rev_q[i]-base)/base*100.0) if base>0 else 0.0)
    rev_cv_recent = _safe_cv(rev_yoy_recent) if len(rev_yoy_recent) >= 2 else None
    stability_score_recent = None if rev_cv_recent is None else _neg(rev_cv_recent, 0.3, 1.5)

    # Moat (recent): CV of quarterly GM% (5 points)
    gm_q = [(gp_q[i]/rev_q[i]*100.0) if rev_q[i]>0 else 0.0 for i in range(len(rev_q))]
    gm_cv_recent = _safe_cv(gm_q) if len(gm_q) >= 3 else None
    moat_score_recent = None if gm_cv_recent is None else _neg(gm_cv_recent, 0.05, 0.25)

    # R&D intensity (recent TTM %)
    rd_intensity_recent = (rd_ttm/rev_ttm*100.0) if rev_ttm>0 else None

    # Investment ratio (recent TTM %)
    invest_recent = (capex_ttm/ocf_ttm*100.0) if ocf_ttm>0 else None

    # ---------- ANNUAL (HISTORICAL) ----------
    # Growth (long): 3y CAGR using 4 annual years y0..y3
    rev_growth_long = None
    if ('Total Revenue' in income_y.index) and (income_y.shape[1] >= 4):
        r0 = _to_float(income_y.loc['Total Revenue'].iloc[0])
        r3 = _to_float(income_y.loc['Total Revenue'].iloc[3])
        if r3 > 0:
            rev_growth_long = ((r0/r3)**(1/3) - 1) * 100.0

    # Stability (long): CV of annual YoY across 3 intervals
    stability_score_long = None
    if ('Total Revenue' in income_y.index) and (income_y.shape[1] >= 4):
        ry = [_to_float(income_y.loc['Total Revenue'].iloc[i], 0.0) for i in range(4)]
        yoy = [((ry[i]-ry[i+1])/ry[i+1]*100.0) if ry[i+1]>0 else 0.0 for i in range(3)]
        cv_long = _safe_cv(yoy) if len(yoy) >= 2 else None
        stability_score_long = None if cv_long is None else _neg(cv_long, 0.3, 1.5)

    # Moat (long): CV of annual GM% over 4 years
    moat_score_long = None
    if all(lbl in income_y.index for lbl in ['Gross Profit','Total Revenue']) and (income_y.shape[1] >= 4):
        gp_y = [_to_float(income_y.loc['Gross Profit'].iloc[i], 0.0) for i in range(4)]
        rv_y = [_to_float(income_y.loc['Total Revenue'].iloc[i], 0.0) for i in range(4)]
        gm_y = [(gp_y[i]/rv_y[i]*100.0) if rv_y[i]>0 else 0.0 for i in range(4)]
        gm_cv_long = _safe_cv(gm_y) if len(gm_y) >= 2 else None
        moat_score_long = None if gm_cv_long is None else _neg(gm_cv_long, 0.05, 0.25)

    # R&D intensity (long): average of last 3 annual R&D%
    rd_intensity_long = None
    if all(lbl in income_y.index for lbl in ['Research And Development','Total Revenue']) and (income_y.shape[1] >= 3):
        rds = []
        for i in range(3):
            rev_i = _to_float(income_y.loc['Total Revenue'].iloc[i], 0.0)
            rd_i  = _to_float(income_y.loc['Research And Development'].iloc[i], 0.0)
            rds.append((rd_i/rev_i*100.0) if rev_i>0 else None)
        rds = [x for x in rds if x is not None]
        rd_intensity_long = sum(rds)/len(rds) if rds else None

    # Investment ratio (long): median of last 3 annual CapEx/OCF
    invest_long = None
    if (cashflow_y is not None) and all(lbl in cashflow_y.index for lbl in ['Capital Expenditure','Operating Cash Flow']) and (cashflow_y.shape[1] >= 3):
        vals = []
        for i in range(3):
            cap = abs(_to_float(cashflow_y.loc['Capital Expenditure'].iloc[i], 0.0))
            ocf = _to_float(cashflow_y.loc['Operating Cash Flow'].iloc[i], 0.0)
            vals.append((cap/ocf*100.0) if ocf>0 else None)
        invest_long = _median(vals)

    # ---------- BLEND ----------
    # Growth (use raw %)
    try:
        rev_growth = _blend(rev_ttm_growth_recent, rev_growth_long, ALPHA_GROWTH_RECENT)
    except Exception as e:
        import pdb; pdb.set_trace()

    # Stability & Moat are already scores in 0..1 (after _neg), blend scores directly
    stability_score = _blend(stability_score_recent, stability_score_long, ALPHA_STAB_RECENT)
    moat_score      = _blend(moat_score_recent,      moat_score_long,      ALPHA_MOAT_RECENT)
    

    # R&D intensity (%), Investment ratio (%)
    rd_intensity = _blend(rd_intensity_recent, rd_intensity_long, ALPHA_RD_RECENT)
    invest_ratio = _blend(invest_recent,       invest_long,       ALPHA_INV_RECENT)
    
    # ---------- Core items unchanged ----------
    fcf4 = _series_recentN(cashflow_q, 'Free Cash Flow', 4)
    rev4 = _series_recentN(income_q,   'Total Revenue', 4)
    fcf_w = _weighted_recent(fcf4, [1.0, 0.75, 0.50, 0.25])
    rev_w = _weighted_recent(rev4,     [1.0, 0.75, 0.50, 0.25])
    fcf_margin = (fcf_w/rev_w*100.0) if rev_w>0 else 0.0

    if ('Current Assets' in balance_q.index) and ('Current Liabilities' in balance_q.index):
        ca = _to_float(balance_q.loc['Current Assets'].iloc[0]); cl = _to_float(balance_q.loc['Current Liabilities'].iloc[0])
        curr_ratio = (ca/cl) if (cl and cl!=0) else _to_float(info_dict.get('currentRatio', 1.0))
    else:
        curr_ratio = _to_float(info_dict.get('currentRatio', 1.0))

    if ('Total Debt' in balance_q.index) and ('Stockholders Equity' in balance_q.index):
        td = _to_float(balance_q.loc['Total Debt'].iloc[0], 0.0); se = _to_float(balance_q.loc['Stockholders Equity'].iloc[0], 0.0)
        debt_eq = (td / se) if (se > 0 and not math.isnan(td)) else _to_float(info_dict.get('debtToEquity', 0.0)/100)
    else:
        debt_eq = _to_float(info_dict.get('debtToEquity', 0.0))/100.0

    return (rev_growth, fcf_margin, debt_eq, curr_ratio,
            stability_score, moat_score, rd_intensity, invest_ratio)

In [94]:
# ---------------- main score ----------------
def buy_score(info_dict, income_q, cashflow_q, balance_q, income_y, cashflow_y, importance_factors):
    # Profitability snapshot (convert decimals → %)
    gm  = _to_pct(info_dict.get('grossMargins', 0))
    om  = _to_pct(info_dict.get('operatingMargins', 0))
    roe = _to_pct(info_dict.get('returnOnEquity', 0))

    # Valuation
    fpe = _to_float(info_dict.get('forwardPE', 0))
    peg = _to_float(info_dict.get('trailingPegRatio', 0))

    (rev_g, fcf_margin, debt_eq, curr_ratio,
     stability, moat, rd_intensity, invest_ratio) = extract_from_statements(
        info_dict, income_q, cashflow_q, balance_q, income_y, cashflow_y
    )

    # Core subscores
    try:
        growth = _pos(rev_g, 5, 40)
    except:
        import pdb; pdb.set_trace()

    profitability = (
        _pos(gm, 40, 70) +
        _pos(om, 15, 45) +
        _pos(roe, 10, 40) +
        _pos(fcf_margin, 5, 35)
    ) / 4.0

    if peg <= 0:
        valuation = _neg(fpe, 12, 45)
    else:
        valuation = (_neg(fpe, 12, 45) + _neg(peg, 1.0, 3.0)) / 2.0

    safety = (
        _neg(debt_eq, 0.0, 1.0) +
        _pos(curr_ratio, 1.0, 3.0)
    ) / 2.0

    # New subscores (using annuals)

    stability = 0.5 if pd.isna(stability) else stability
    moat      = 0.5 if pd.isna(moat) else moat
    rd_score  = 0 if pd.isna(rd_intensity) else _pos(rd_intensity, 5, 22)
    invest_sc = 0.5 if pd.isna(invest_ratio) else _pos(invest_ratio, 10, 45)
    
    score01 = (
        importance_factors['growth']*growth +
        importance_factors['profitability']*profitability +
        importance_factors['valuation']*valuation +
        importance_factors['safety']*safety +
        importance_factors['stability']*stability +
        importance_factors['moat']*moat +
        importance_factors['rd_score']*rd_score +
        importance_factors['invest_score']*invest_sc
    )
    score = round(100*score01, 1)

    metric_vals = [
        gm, om, roe, fpe, peg,
        rev_g, fcf_margin, debt_eq, curr_ratio,
        rd_intensity, invest_ratio,
        growth, profitability, valuation, safety,
        stability, moat, rd_score, invest_sc,
        score
    ]
    return score, metric_vals

def score_label(score):
    if score >= 75: return "BUY (High Conviction)"
    if score >= 60: return "ACCUMULATE / HOLD"
    return "AVOID / WATCHLIST"

def rank_stocks(bundles, importance_factors):
    """
    bundles: [{'fund': info_dict,
               'inc_q': incQ, 'cf_q': cfQ, 'bs_q': bsQ,
               'inc_y': incY, 'cf_y': cfY}, ...]
    """
    out = []
    ticker_df = pd.DataFrame(columns=["Company", "Growth Margin", "Operating Margin", "ROE", "Forward P/E", "PEG", "Revenue TTM Growth", "FCF Margin", "Debt/Equity", "Current Ratio", "R&D Intensity", "Investment Ratio", "Growth", "Profitability", "Valuation", "Safety", "Stability Score (Rev- Var)", "Moat Score (G.Margin- Var)", "R&D Score", "Investment Score", "Score"])
    for b in bundles:
        name = b['fund'].get('longName') or b['fund'].get('shortName') or b['fund'].get('symbol') or 'Unknown'
        sc, metric_vals = buy_score(b['fund'], b['inc_q'], b['cf_q'], b['bs_q'], b['inc_y'], b['cf_y'], importance_factors)
        ticker_df.loc[len(ticker_df)] = [name] + metric_vals
        out.append((name, sc, score_label(sc)))
    return sorted(out, key=lambda x: x[1], reverse=True), ticker_df
# --- end ----------------------------------------------------------------------


In [106]:
# --- BLENDED RECENT + HISTORICAL (add/replace inside extract_from_statements) --

ALPHA_GROWTH_RECENT   = 0.60  # TTM vs 3y CAGR
ALPHA_STAB_RECENT     = 0.60  # quarterly vs annual CV score
ALPHA_MOAT_RECENT     = 0.60
ALPHA_RD_RECENT       = 0.70  # TTM vs annual avg
ALPHA_INV_RECENT      = 0.70

importance_factors = {'growth': 0.25, 'profitability': 0.25, 'valuation': 0.25, 'safety': 0.25, 'stability': 0, 'moat': 0 , 'rd_score': 0, 'invest_score': 0}


In [107]:
# rankings = rank_stocks(bundles)
rankings, ticker_df = rank_stocks(bundles, importance_factors)

/var/folders/mc/s2w117k11cj56552qbkf28zw0000gn/T/ipykernel_31638/3397957110.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ticker_df.loc[len(ticker_df)] = [name] + metric_vals
/var/folders/mc/s2w117k11cj56552qbkf28zw0000gn/T/ipykernel_31638/3397957110.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ticker_df.loc[len(ticker_df)] = [name] + metric_vals
/var/folders/mc/s2w117k11cj56552qbkf28zw0000gn/T/ipykernel_31638/3397957110.py:85: FutureWarning: The behavior of DataFrame concatenatio

In [108]:
rankings

[('NVIDIA Corporation', 86.1, 'BUY (High Conviction)'),
 ('Arista Networks Inc', 81.2, 'BUY (High Conviction)'),
 ('SK hynix Inc.', 72.3, 'ACCUMULATE / HOLD'),
 ('Taiwan Semiconductor Manufacturing Company Limited',
  70.4,
  'ACCUMULATE / HOLD'),
 ('Waaree Energies Limited', 60.6, 'ACCUMULATE / HOLD'),
 ('Super Micro Computer, Inc.', 60.3, 'ACCUMULATE / HOLD'),
 ('Analog Devices, Inc.', 57.9, 'AVOID / WATCHLIST'),
 ('Novo Nordisk A/S', 57.9, 'AVOID / WATCHLIST'),
 ('Adobe Inc.', 55.9, 'AVOID / WATCHLIST'),
 ('Eli Lilly and Company', 53.6, 'AVOID / WATCHLIST'),
 ('Photronics, Inc.', 51.9, 'AVOID / WATCHLIST'),
 ('Alphabet Inc.', 51.6, 'AVOID / WATCHLIST'),
 ('Lam Research Corporation', 51.0, 'AVOID / WATCHLIST'),
 ('Marvell Technology, Inc.', 50.8, 'AVOID / WATCHLIST'),
 ('Broadcom Inc.', 49.3, 'AVOID / WATCHLIST'),
 ('KLA Corporation', 49.0, 'AVOID / WATCHLIST'),
 ('ASML Holding N.V.', 48.7, 'AVOID / WATCHLIST'),
 ('Micron Technology, Inc.', 47.8, 'AVOID / WATCHLIST'),
 ('Voltamp Tran

In [109]:
ticker_df.sort_values(by='Score', ascending=False)

,Company,Growth Margin,Operating Margin,ROE,Forward P/E,PEG,Revenue TTM Growth,FCF Margin,Debt/Equity,Current Ratio,R&D Intensity,Investment Ratio,Growth,Profitability,Valuation,Safety,Stability Score (Rev- Var),Moat Score (G.Margin- Var),R&D Score,Investment Score,Score
37,NVIDIA Corporation,69.85,60.84,109.42,45.21,1.00,61.06,41.66,0.11,4.21,11.65,6.07,1.00,1.00,0.50,0.95,0.53,0.75,0.39,0.00,86.10
11,Arista Networks Inc,64.24,44.73,33.64,15.81,2.40,31.63,47.54,0.00,3.33,14.09,1.75,0.76,0.90,0.59,1.00,0.89,1.00,0.53,0.00,81.20
40,SK hynix Inc.,53.91,41.44,39.28,8.44,0.71,27.41,18.59,0.28,1.78,7.64,82.34,0.64,0.69,1.00,0.56,0.00,0.51,0.16,1.00,72.30
23,Taiwan Semiconductor Manufacturing Company Lim...,58.98,50.58,34.66,36.50,1.52,32.05,26.99,0.19,2.37,7.57,67.65,0.77,0.80,0.50,0.75,0.38,0.98,0.15,1.00,70.40
47,Waaree Energies Limited,32.51,19.23,33.70,23.64,0.00,59.92,1.42,0.16,1.50,0.00,78.83,1.00,0.23,0.65,0.55,0.67,0.25,0.00,1.00,60.60
8,"Super Micro Computer, Inc.",11.06,3.97,17.90,11.39,0.00,29.19,9.19,0.76,5.25,3.06,7.35,0.69,0.10,1.00,0.62,0.74,0.41,0.00,0.00,60.30
42,Novo Nordisk A/S,84.26,43.52,79.17,13.01,1.48,18.67,19.24,0.59,0.78,14.57,45.23,0.39,0.86,0.86,0.20,1.00,1.00,0.56,1.00,57.90
21,"Analog Devices, Inc.",60.15,28.55,5.65,31.57,0.82,18.27,35.06,0.25,2.32,15.65,13.82,0.38,0.53,0.70,0.70,0.00,1.00,0.63,0.11,57.90
39,Adobe Inc.,89.14,36.29,52.88,17.20,1.00,10.78,38.77,0.56,1.02,17.99,2.86,0.17,0.93,0.92,0.22,1.00,1.00,0.76,0.00,55.90
48,Eli Lilly and Company,82.64,45.80,86.29,36.43,0.90,29.28,0.95,2.18,1.28,23.23,98.91,0.69,0.75,0.63,0.07,0.50,1.00,1.00,1.00,53.60


In [114]:
stock= yf.Ticker("WAAREEENER.BO")

In [116]:
stock.quarterly_incomestmt

,2025-06-30,2025-03-31,2024-12-31,2024-06-30,2024-03-31
Tax Effect Of Unusual Items,0.00,-13788361.87,0.00,0.00,896048477.88
Tax Rate For Calcs,0.18,0.24,0.27,0.24,0.28
Normalized EBITDA,11686700000.00,9981690000.00,8097050000.00,6399950000.00,4445780000.00
Total Unusual Items,0.00,-58000000.00,0.00,0.00,3217080000.00
Total Unusual Items Excluding Goodwill,0.00,-58000000.00,0.00,0.00,3217080000.00
Net Income From Continuing Operation Net Minority Interest,7452000000.00,6189120000.00,4925870000.00,3941500000.00,4615330000.00
Reconciled Depreciation,1820600000.00,1533700000.00,890470000.00,757650000.00,751510000.00
Reconciled Cost Of Revenue,30016500000.00,28465370000.00,24396980000.00,25823600000.00,23468010000.00
EBITDA,11686700000.00,9923690000.00,8097050000.00,6399950000.00,7662860000.00
EBIT,9866100000.00,8389990000.00,7206580000.00,5642300000.00,6911350000.00
